In [ ]:
!pip install -U -q scikit-image matplotlib numpy

In [24]:
from skimage import io as skio ## Image input/output
from skimage.color import label2rgb ## For plotting: render each mask with its own color

import numpy as np
from glob import glob

from matplotlib import pyplot as plt
import matplotlib

import warnings
warnings.filterwarnings('ignore')

In [23]:
def TupleRounder(coords:tuple):

    x = int(np.round(coords[0],0))
    y = int(np.round(coords[1],0))

    return (x,y)

def MaskSelector(masks:np.array,coords:list):

    NewArray = np.zeros(masks.shape)

    instances = []
    
    for coordinate in coords:

        x,y = coordinate[0],coordinate[1]

        instance = masks[y,x]
        
        instances.append(instance)

    instances = np.unique(instances)
        
    for i in instances:

        if i == 0: continue

        singleton = (masks == i)*i

        NewArray += singleton
        
    return np.uint16(NewArray)

In [5]:
segmentations = glob('/Users/reyesmatte/Desktop/Reuter_2025/Watershed_example/*watershed.tif')

print(len(segmentations))

1


### Interactively select masks that will be considered for further analyses

The segmentation image will open in a separate floating window, where you can interactively mark individual instances for the plaques that **will be kept for further analysis**

Here are the controls:

- To mark a plaque, left-click or press the spacebar
- To un-mark a plaque, right-click
- To go to the next image, press Enter


In [20]:
matplotlib.use('tkagg')

rois = []
processed_masks = []

for masks in segmentations:

    image = skio.imread(masks)

    if image[0,0] > 0:

        image -= image[0,0]
    
    fig, ax=plt.subplots()
    ax.imshow(label2rgb(image))
    yroi = plt.ginput(n=-1,timeout=0,show_clicks=True,mouse_add=1,mouse_pop=3,mouse_stop=2)

    coords = [TupleRounder(roi) for roi in yroi]
    
    processed_masks.append(MaskSelector(skio.imread(masks),coords=coords))
    
    rois.append(coords)

In [ ]:
assert len(segmentations) == len(processed_masks)
segmentations = [replace('_watershed.tif','_selected.tif') for segfile in segmentations]

for name,mask in zip(segmentations,processed_masks):
    
    skio.imwrite(name,mask)